In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
file=open("/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/case-03/original/T3.java")
og=file.read()

In [67]:
print(og)

import java.util.Scanner;

public class T3 {
	public static void main(String[] args) {
		Scanner input = new Scanner(System.in);

		// Prompt the user to enter weight in pounds
		System.out.print("Enter weight in pounds: ");
		double weight = input.nextDouble();

		// Prompt the user to enter height
		System.out.print("Enter feet: ");
		double feet = input.nextDouble();
		System.out.print("Enter inches: ");
		double inches = input.nextDouble();

		double height = feet * 12 + inches;

		// Compute BMI
		double bmi = weight * 0.45359237 / ((height * 0.0254) * (height * 0.0254));

		// Display result
		System.out.println("BMI is " + bmi);
		if (bmi < 18.5)
			System.out.println("Underweight");
		else if (bmi < 25)
			System.out.println("Normal");
		else if (bmi < 30)
			System.out.println("Overweight");
		else
			System.out.println("Obese");
	}

}



In [68]:
!pip install javalang


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [69]:
import javalang


In [70]:
tree = javalang.parse.parse(og)

In [71]:
for path,node in tree:
  print(node)


CompilationUnit(imports=[Import(path=java.util.Scanner, static=False, wildcard=False)], package=None, types=[ClassDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=[LocalVariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=ClassCreator(arguments=[MemberReference(member=in, postfix_operators=[], prefix_operators=[], qualifier=System, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=Scanner, sub_type=None)), name=input)], modifiers=set(), type=ReferenceType(arguments=None, dimensions=[], name=Scanner, sub_type=None)), StatementExpression(expression=MethodInvocation(arguments=[Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="Enter weight in pounds: ")], member=print, postfix_operators=[], prefix_operators=[], qualifier=System.out, selectors=[], typ

In [74]:
#number of methods and their names and return type

for path,node in tree.filter(javalang.tree.MethodDeclaration):
  print(node.name)
  if(node.return_type):
    print(node.return_type.name)
  else:
    print(node.return_type)

main
None


In [75]:
#number of variables and their names

for path,node in tree.filter(javalang.tree.VariableDeclaration):
  for variable_declarator in node.declarators:
    print(variable_declarator.name)

input
weight
feet
inches
height
bmi


In [76]:
#number of localvariables and their names

for path,node in tree.filter(javalang.tree.LocalVariableDeclaration):
  print(node.type.name )
  for variable_declarator in node.declarators:
    print(variable_declarator.name)

Scanner
input
double
weight
double
feet
double
inches
double
height
double
bmi


In [77]:
for path,node in tree:

  if isinstance(node, javalang.tree.ForStatement):
          print(node)
  if isinstance(node, javalang.tree.DoStatement):
          print(node)
  if isinstance(node, javalang.tree.WhileStatement):
          print(node)

In [81]:
for path,node in tree.filter(javalang.tree.IfStatement):
  print(node.else_statement)


IfStatement(condition=BinaryOperation(operandl=MemberReference(member=bmi, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), operandr=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=25), operator=<), else_statement=IfStatement(condition=BinaryOperation(operandl=MemberReference(member=bmi, postfix_operators=[], prefix_operators=[], qualifier=, selectors=[]), operandr=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value=30), operator=<), else_statement=StatementExpression(expression=MethodInvocation(arguments=[Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="Obese")], member=println, postfix_operators=[], prefix_operators=[], qualifier=System.out, selectors=[], type_arguments=None), label=None), label=None, then_statement=StatementExpression(expression=MethodInvocation(arguments=[Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[]

In [86]:
for path,node in tree:
  if isinstance(node,(javalang.tree.IfStatement,javalang.tree.MethodDeclaration)):
    print(node)

MethodDeclaration(annotations=[], body=[LocalVariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=ClassCreator(arguments=[MemberReference(member=in, postfix_operators=[], prefix_operators=[], qualifier=System, selectors=[])], body=None, constructor_type_arguments=None, postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], type=ReferenceType(arguments=None, dimensions=None, name=Scanner, sub_type=None)), name=input)], modifiers=set(), type=ReferenceType(arguments=None, dimensions=[], name=Scanner, sub_type=None)), StatementExpression(expression=MethodInvocation(arguments=[Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="Enter weight in pounds: ")], member=print, postfix_operators=[], prefix_operators=[], qualifier=System.out, selectors=[], type_arguments=None), label=None), LocalVariableDeclaration(annotations=[], declarators=[VariableDeclarator(dimensions=[], initializer=MethodInvocation

In [ ]:
print(og)

import java.util.Scanner;

public class T7 {
	public static void main(String[] args) {
		Scanner input = new Scanner(System.in);

		System.out.print("Enter a 4 by 4 matrix row by row: ");
		double[][] m = new double[4][4];

		for (int i = 0; i < 4; i++)
			for (int j = 0; j < 4; j++)
				m[i][j] = input.nextDouble();

		System.out.print("Sum of the elements in the major diagonal is " + sumMajorDiagonal(m));
	}

	public static double sumMajorDiagonal(double[][] m) {
		double sum = 0;

		for (int i = 0; i < m.length; i++)
			sum += m[i][i];

		return sum;
	}

}



In [ ]:
li=list(javalang.tokenizer.tokenize(og))

In [ ]:
print(li[0])

Modifier "public" line 2, position 1


In [ ]:
#easy to fetch number of identifiers 

In [ ]:
for elem in li:
  if str(elem).split(" ")[0]=='Identifier':
    print(str(elem).split(" ")[1])

In [ ]:
for elem in li:
  print(str(elem).split(" ")[0],str(elem).split(" ")[1])

In [52]:
class CodeAnalyzer:
    def __init__(self, code):
        self.code = code
        self.tree = javalang.parse.parse(self.code)
        self.method_name = []
        self.method_return_type = []
        self.loops = []
        self.controlStruct =[]
        self.nesting_depth = 0
        self.variable_name=[]
        self.variable_datatype=[]

    def analyze_method(self):
        for path,node in self.tree.filter(javalang.tree.MethodDeclaration):
          self.method_name.append(node.name)
          if(node.return_type):
            self.method_return_type.append(node.return_type.name)
          else:
            self.method_return_type.append(node.return_type)        

    def analyze_variables(self):
        for path,node in self.tree.filter(javalang.tree.VariableDeclaration):
          self.variable_datatype.append(node.type.name)
          for variable_declarator in node.declarators:
            self.variable_name.append(variable_declarator.name)

    def analyze_loops(self):
        for path,node in self.tree:
          if isinstance(node, javalang.tree.ForStatement):
                  self.loops.append('For')
          if isinstance(node, javalang.tree.DoStatement):
                  self.loops.append('Do')
          if isinstance(node, javalang.tree.WhileStatement):
                  self.loops.append('While')

    def analyze_control_structures(self):
        for path,node in self.tree:
          if isinstance(node, javalang.tree.IfStatement):
                  self.loops.append('If')
          if isinstance(node, javalang.tree.SwitchStatement):
                  self.loops.append('Switch')
            
    def analyze_nesting_depth(self):
      pass

    def analyze_code(self):
        # Analyze the code and populate the variables accordingly
        self.analyze_method()
        self.number_of_methods = len(self.method_name)

        self.analyze_loops()
        self.no_of_loops = len(self.loops)

        self.analyze_control_structures()
        self.no_of_controlStruct = len(self.controlStruct)

        self.analyze_variables()
        self.no_of_variables=len(self.variable_name)

        self.analyze_nesting_depth()
        





In [56]:
original_code = CodeAnalyzer(og)

In [57]:
original_code.analyze_code()

In [58]:
original_code.method_name

['main', 'sumMajorDiagonal']